In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import os
home_path = os.path.expanduser("~") + '/'
from info_params import get_default_hparams
from utils.load_data import *

OpenCV is built with OpenMP support. This usually results in poor performance. For details, see https://github.com/tensorpack/benchmarks/blob/master/ImageNet/benchmark-opencv-resize.py


In [3]:
def get_data(hps):
    dfX, df_next_deltaClose = load_data_seq(hps)

    segment, next_segment, target_one_hot = segment_seq(dfX, df_next_deltaClose, hps)

    train_segment, test_segment, _, _, train_target_one_hot, test_target_one_hot = \
        train_test_split(segment, next_segment, target_one_hot, hps)

    return train_segment, test_segment, train_target_one_hot, test_target_one_hot

In [4]:
hps = get_default_hparams()
hps.data_file_name = home_path + "Dropbox/thesis/data/processed_data.csv"
hps.attributes_normalize_mean = ['delta_Open','delta_High','delta_Low','delta_Close','delta_Volume BTC','delta_Volume USD','Spread High-Low','Spread Close-Open','MA_Close_24','MA_V_BTC_24','MA_Close_240','MA_V_BTC_240']
hps.is_concat = False
hps.Tau = 1
hps.T = 1
hps.C = 2
hps.D = (1+hps.is_concat) * len(hps.attributes_normalize_mean)
hps.is_differencing = False
hps.lag_time = 1
hps.N_train_seq = 10000
hps.normalize_data = 'default'
hps.normalize_data_idx = True

In [5]:
X_train, X_test, train_target_one_hot, test_target_one_hot = get_data(hps)
X_train = np.reshape(X_train, newshape=[-1, hps.T * hps.D])
X_test = np.reshape(X_test, newshape=[-1, hps.T * hps.D])

y_train = np.argmax(train_target_one_hot, axis=-1)
y_train = np.reshape(y_train, newshape=[-1])

y_test = np.argmax(test_target_one_hot, axis=-1)
y_test = np.reshape(y_test, newshape=[-1])

Missing Normalization


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [7]:
clf = RandomForestClassifier(max_depth =hps.D, random_state=2302, n_estimators=127, criterion='entropy', warm_start = True)

In [8]:
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=12, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=127, n_jobs=None,
            oob_score=False, random_state=2302, verbose=0, warm_start=True)

# Train metrics

In [9]:
y_train_hat = clf.predict(X_train)
from sklearn.metrics import classification_report, confusion_matrix

cm = confusion_matrix(y_train, y_train_hat)
rp = classification_report(y_train, y_train_hat)

print(cm)
print(rp)

[[3828 1000]
 [ 629 4543]]
              precision    recall  f1-score   support

           0       0.86      0.79      0.82      4828
           1       0.82      0.88      0.85      5172

   micro avg       0.84      0.84      0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000



# Test metrics

In [10]:
y_pred = clf.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)
rp = classification_report(y_test, y_pred)
acc = accuracy_score(y_true=y_test, y_pred = y_pred)


print(cm)
print(rp)
print("accuracy :{}".format(acc))

[[ 681 1657]
 [ 554 1880]]
              precision    recall  f1-score   support

           0       0.55      0.29      0.38      2338
           1       0.53      0.77      0.63      2434

   micro avg       0.54      0.54      0.54      4772
   macro avg       0.54      0.53      0.51      4772
weighted avg       0.54      0.54      0.51      4772

accuracy :0.5366722548197821


# Test 1000 first

In [11]:
N = 1000
y_pred = clf.predict(X_test[0:N])
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

cm = confusion_matrix(y_test[0:N], y_pred)
rp = classification_report(y_test[0:N], y_pred)
acc = accuracy_score(y_true=y_test[0:N], y_pred = y_pred)
print(cm)
print(rp)
print("accuracy :{}".format(acc))

[[245 244]
 [187 324]]
              precision    recall  f1-score   support

           0       0.57      0.50      0.53       489
           1       0.57      0.63      0.60       511

   micro avg       0.57      0.57      0.57      1000
   macro avg       0.57      0.57      0.57      1000
weighted avg       0.57      0.57      0.57      1000

accuracy :0.569


# Save Model and predict:
- File model: 24.6 Mb

In [12]:
#Saving model
from sklearn.externals import joblib
joblib.dump(clf, 'checkpoint/Random_Forest_sklearn.joblib')

['checkpoint/Random_Forest_sklearn.joblib']

# Visualized

In [16]:
from sklearn.tree import export_graphviz
from IPython import display
from sklearn.ensemble import RandomForestRegressor


str_tree = export_graphviz(clf, 
   out_file=None, 
   filled=True,        
   special_characters=True, 
   rotate=True)

display.display(str_tree)

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.